In [116]:
import math
import numpy as np
import pandas as pd
from typing import Tuple, List, Optional, Any, Dict


# ML / Torch imports (keep but user must have torch installed)
import torch
import torch.nn as nn
import joblib


# these packages were used by you before; keep consistent
import talib
import ta
import ccxt

In [117]:
symbol = "BTC/USDT"
market_type = "spot"           # 'spot' implemented here. For futures you'd change code.
model_timeframe = "15m"
price_timeframe = "1s"
limit = 300
window_size = 10

hold_factor = 0.2

FEATURES = ['RSI', 'EMA12', 'EMA26', 'MACD', 'Signal', 'Histogram', 'DEMA9', 'SMA', 'TSI', '%K', '%D']

SAVED_MODEL = "greg_tech_5.pth"
SCALER_FILE = "scaler_15m.pkl"

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# MSB/OB hyperparameters (tweak)
ZIGZAG_LEN = 9
FIB_FACTOR = 0.33
OB_LOOKBACK = 10

In [118]:
# -------------------------
# MODEL
# -------------------------
class CryptoLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, num_layers=4, output_dim=3):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        return self.fc(out)

model = CryptoLSTM(input_dim=len(FEATURES), hidden_dim=128, num_layers=4, output_dim=3)
model.load_state_dict(torch.load(SAVED_MODEL, map_location=DEVICE))
model.to(DEVICE); model.eval()
scaler = joblib.load(SCALER_FILE)


In [119]:
def compute_tsi(close, r1=25, r2=13):
    delta = close.diff()
    ema1 = delta.ewm(span=r1, adjust=False).mean()
    ema2 = ema1.ewm(span=r2, adjust=False).mean()

    abs_delta = delta.abs()
    abs_ema1 = abs_delta.ewm(span=r1, adjust=False).mean()
    abs_ema2 = abs_ema1.ewm(span=r2, adjust=False).mean()

    tsi = 100 * (ema2 / abs_ema2)
    return tsi


In [120]:
import ta
def compute_indicators(df):
    out = {}
    out["RSI"] = ta.momentum.RSIIndicator(df["close"], window=14).rsi()
    out["EMA12"] = df["close"].ewm(span=12, adjust=False).mean()
    out["EMA26"] = df["close"].ewm(span=26, adjust=False).mean()
    out["MACD"] = out["EMA12"] - out["EMA26"]
    out["Signal"] = out["MACD"].ewm(span=9, adjust=False).mean()
    out["Histogram"] = out["MACD"] - out["Signal"]
    out["DEMA9"] = talib.DEMA(df["close"].values, timeperiod=9)
    sma_window = 3
    out['SMA'] = ta.trend.sma_indicator(df['close'], window=sma_window)
    out['TSI'] = compute_tsi(df['close'])
    period = 14
    smooth_k = 3
    smooth_d = 3

    lowest_low = df["low"].rolling(period).min()
    highest_high = df["high"].rolling(period).max()

    out["%K"] = 100 * (df["close"] - lowest_low) / (highest_high - lowest_low)
    out["%K"] = out["%K"].rolling(smooth_k).mean()
    out["%D"] = out["%K"].rolling(smooth_d).mean()
    
    feat_df = pd.DataFrame(out, index=df.index)
    return feat_df


In [121]:
# -------------------------
# CCXT EXCHANGE SETUP
# -------------------------
exchange = ccxt.binance({
    'enableRateLimit': True,
})

In [122]:
# -------------------------
# FETCH DATA
# -------------------------
def fetch_latest_ohlcv(symbol, timeframe, limit):
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)
    df = pd.DataFrame(ohlcv, columns=['timestamp','open','high','low','close','volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df.columns = [c.lower() for c in df.columns]
    return df


In [123]:
# -------------------------
# Prediction
# -------------------------
def predict_label_from_window(window_features):
    flat = scaler.transform(window_features)
    x = torch.tensor(flat, dtype=torch.float32).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        logits = model(x)
        probs = torch.softmax(logits, dim=1).cpu().numpy()[0]
        probs[1] *= hold_factor
        probs = probs / probs.sum()
        pred = int(np.argmax(probs))
    return pred  # 0=down 1=hold 2=up


In [124]:
pred = predict_label_from_window(np.random.rand(window_size, len(FEATURES)))
print(pred)

1


In [125]:
def detect_market_structure(df: pd.DataFrame, zigzag_len: int = ZIGZAG_LEN, fib_factor: float = FIB_FACTOR) -> pd.DataFrame:
    """Detect trend, MSB (market structure break), and record swing highs/lows.


    Adds columns:
    - to_up, to_down (bool)
    - trend (1 or -1)
    - trend_change (0 or non-zero)
    - msb (0/1/-1) where 1 means bullish MSB, -1 bearish MSB
    - swing_high_val, swing_high_idx, swing_low_val, swing_low_idx
    """
    n = len(df)
    highs = df['high']
    lows = df['low']
    closes = df['close']


    df = df.copy()
    df['to_up'] = highs >= highs.rolling(zigzag_len, min_periods=1).max()
    df['to_down'] = lows <= lows.rolling(zigzag_len, min_periods=1).min()


    trend = [1]
    swing_highs = [np.nan]
    swing_high_idx = [np.nan]
    swing_lows = [np.nan]
    swing_low_idx = [np.nan]
    msb = [0]
    
    # We'll track recent highs/lows similar to Pine Script arrays
    recent_high_vals = []
    recent_high_idxs = []
    recent_low_vals = []
    recent_low_idxs = []


    for i in range(1, n):
        prev_trend = trend[-1]
        if prev_trend == 1 and df['to_down'].iat[i]:
            cur_trend = -1
            # record swing high (previous highest in window)
            look_back = min(zigzag_len, i)
            h_val = highs.iloc[i - look_back:i].max()
            h_idx = highs.iloc[i - look_back:i].idxmax()
            recent_high_vals.append(h_val)
            recent_high_idxs.append(h_idx)
        elif prev_trend == -1 and df['to_up'].iat[i]:
            cur_trend = 1
            # record swing low
            look_back = min(zigzag_len, i)
            l_val = lows.iloc[i - look_back:i].min()
            l_idx = lows.iloc[i - look_back:i].idxmin()
            recent_low_vals.append(l_val)
            recent_low_idxs.append(l_idx)
        else:
            cur_trend = prev_trend
            
        trend.append(cur_trend)


        # Determine MSB using fib_factor heuristic similar to Pine script
        cur_msb = 0
        if cur_trend != prev_trend:
            # If trend flipped, mark msb at this bar
            cur_msb = cur_trend
            # (Pine script had more complex checks — this is a conservative simplification)
        msb.append(cur_msb)


        # mirrors for debug columns
        if recent_high_vals:
            swing_highs.append(recent_high_vals[-1])
            swing_high_idx.append(recent_high_idxs[-1])
        else:
            swing_highs.append(np.nan)
            swing_high_idx.append(np.nan)
        if recent_low_vals:
            swing_lows.append(recent_low_vals[-1])
            swing_low_idx.append(recent_low_idxs[-1])
        else:
            swing_lows.append(np.nan)
            swing_low_idx.append(np.nan)    
            
    df['trend'] = trend
    df['trend_change'] = df['trend'].diff().fillna(0)
    df['msb'] = msb
    df['swing_high_val'] = swing_highs
    df['swing_high_idx'] = swing_high_idx
    df['swing_low_val'] = swing_lows
    df['swing_low_idx'] = swing_low_idx


    return df        

In [126]:
def detect_order_blocks(df: pd.DataFrame, ob_lookback: int = OB_LOOKBACK) -> pd.DataFrame:
    """After MSBs are known, create bullish/bearish OB zones using previous bars.


    Adds columns:
    - bull_ob_low, bull_ob_high (nullable floats)
    - bear_ob_low, bear_ob_high
    """
    df = df.copy()
    df['bull_ob_low'] = np.nan
    df['bull_ob_high'] = np.nan
    df['bear_ob_low'] = np.nan
    df['bear_ob_high'] = np.nan
    
    for i in range(len(df)):
        if df['msb'].iat[i] == 1:
            # bullish MSB; OB is a range in the lookback before MSB
            start = max(0, i - ob_lookback)
            ob_low = df['low'].iloc[start:i+1].min()
            ob_high = df['high'].iloc[start:i+1].max()
            df.at[df.index[i], 'bull_ob_low'] = ob_low
            df.at[df.index[i], 'bull_ob_high'] = ob_high
        elif df['msb'].iat[i] == -1:
            start = max(0, i - ob_lookback)
            ob_low = df['low'].iloc[start:i+1].min()
            ob_high = df['high'].iloc[start:i+1].max()
            df.at[df.index[i], 'bear_ob_low'] = ob_low
            df.at[df.index[i], 'bear_ob_high'] = ob_high
    return df

In [127]:
def combine_ml_and_msb(df: pd.DataFrame, ml_preds: List[int], hold_factor: float = 1.0) -> pd.DataFrame:
    """Attach ML predictions and compute confirmation columns.


    ml_preds should align with df.index (same length).
    """
    df = df.copy()
    df['ml_pred'] = ml_preds


    # Adjust hold probability multiplier is model-side but here we just keep preds
    df['confirmed_up'] = (df['ml_pred'] == 2) & (df['msb'] == 1)
    df['confirmed_down'] = (df['ml_pred'] == 0) & (df['msb'] == -1)


    # optional: check price inside OB
    df['in_bull_ob'] = False
    df['in_bear_ob'] = False
    
    for i in range(len(df)):
        close = df['close'].iat[i]
        if not np.isnan(df.get('bull_ob_low').iat[i]):
            if df['bull_ob_low'].iat[i] <= close <= df['bull_ob_high'].iat[i]:
                df.at[df.index[i], 'in_bull_ob'] = True
        if not np.isnan(df.get('bear_ob_low').iat[i]):
            if df['bear_ob_low'].iat[i] <= close <= df['bear_ob_high'].iat[i]:
                df.at[df.index[i], 'in_bear_ob'] = True
                
    # Final signals: default 1 (hold), 2=buy, 0=sell
    df['final_signal'] = 1
    # require both ML and msb and price in OB (loose rule: allow without OB if OB missing)
    for i in range(len(df)):
        if df['confirmed_up'].iat[i]:
            if df['in_bull_ob'].iat[i] or np.isnan(df.get('bull_ob_low').iat[i]):
                df.at[df.index[i], 'final_signal'] = 2
        if df['confirmed_down'].iat[i]:
            if df['in_bear_ob'].iat[i] or np.isnan(df.get('bear_ob_low').iat[i]):
                df.at[df.index[i], 'final_signal'] = 0


    return df            

In [128]:
def sliding_predict(df_ohlcv: pd.DataFrame, window_size: int = window_size) -> pd.Series:
    """
    Run windowed predictions across a historical df_ohlcv that contains
    'open,high,low,close,volume'.

    Returns:
        A pd.Series of predictions aligned with df_ohlcv.index.
        (First `window_size` entries will be NaN because there's
         not enough history to form a full window.)
    """
    # Compute indicators and drop invalid (NaN) rows
    ind = compute_indicators(df_ohlcv).dropna().copy()

    # Make sure indexes align
    ind = ind.reindex(df_ohlcv.index)
    ind = ind.dropna()  # ensures all features are valid

    preds = pd.Series(np.nan, index=df_ohlcv.index, dtype='float')

    # Ensure no NaNs in features before prediction
    ind = ind.dropna(subset=FEATURES)

    for i in range(window_size, len(ind)):
        # Get last 'window_size' feature rows
        window_features = ind[FEATURES].iloc[i - window_size:i]

        # Skip if the window still has NaNs (safety check)
        if window_features.isnull().values.any():
            continue

        # Scale safely (convert to numpy array)
        X_scaled = scaler.transform(window_features.values)

        # Predict label using your trained model
        pred = predict_label_from_window(X_scaled)

        # Store prediction in the correct timestamp
        preds.iloc[df_ohlcv.index.get_indexer_for([ind.index[i]])[0]] = pred

    # Fill possible tiny gaps (if any)
    preds = preds.fillna(method='ffill').fillna(method='bfill')

    return preds


In [129]:
back_df = fetch_latest_ohlcv(symbol, model_timeframe, 30 + window_size)
feat_df = compute_indicators(back_df).dropna()

    # Run short back-prediction to determine recent trend
preds = [
        predict_label_from_window(feat_df[FEATURES].values[i - window_size:i])
        for i in range(window_size, len(feat_df))
    ]

print(preds)

[1, 1, 1, 1, 1, 1, 1, 2, 1, 0, 1, 1, 0]


In [130]:
def run_backtest(df_ohlcv: pd.DataFrame, final_signals: pd.Series, initial_capital: float = 10000.0) -> Dict[str, Any]:
    """Simple backtest: market orders at next bar open. No fees or slippage.


    final_signals: series aligned with df_ohlcv; values: 0=sell,1=hold,2=buy
    Returns summary dict with trades and final equity.
    """
    equity = initial_capital
    position = 0.0 # positive means long amount in quote currency (asset quantity)
    asset_qty = 0.0
    trades = []
    
    # Convert signals -> actions at next open
    for i in range(len(df_ohlcv) - 1):
        sig = final_signals.iat[i]
        next_open = df_ohlcv['open'].iat[i + 1]
        if np.isnan(sig):
            continue
        # BUY: use all equity to buy asset
        if sig == 2 and equity > 0:
            asset_qty = equity / next_open
            trades.append({'index': df_ohlcv.index[i+1], 'side':'buy', 'price': next_open, 'qty': asset_qty})
            equity = 0.0
            position = 1
        # SELL: liquidate asset
        elif sig == 0 and asset_qty > 0:
            proceeds = asset_qty * next_open
            trades.append({'index': df_ohlcv.index[i+1], 'side':'sell', 'price': next_open, 'qty': asset_qty})
            equity = proceeds
            asset_qty = 0.0
            position = 0
            
    # finalize
    if asset_qty > 0:
        # mark to market at last close
        final_price = df_ohlcv['close'].iat[-1]
        equity = asset_qty * final_price


    return {
        'final_equity': equity,
        'trades': trades,
        'return_pct': (equity - initial_capital) / initial_capital * 100.0
    }        

In [131]:
def build_hybrid_signals(df_ohlcv: pd.DataFrame, model: nn.Module, scaler: Any,
    window_size: int = window_size) -> pd.DataFrame:
    """Produce a dataframe with indicators, ml preds, msb, ob, and final signals.


    Returns full df with columns including 'final_signal'.
    """
    # compute indicators
    ind_df = compute_indicators(df_ohlcv).dropna()
    feat_df = df_ohlcv.join(ind_df, how='inner').dropna()


    # sliding ML predictions
    preds = [
        predict_label_from_window(feat_df[FEATURES].values[i - window_size:i])
        for i in range(window_size, len(feat_df))
    ]

    feat_df['ml_pred'] = [pred for pred in [np.nan]*window_size + preds]


    # detect MSB and OB
    msb_df = detect_market_structure(feat_df)
    ob_df = detect_order_blocks(msb_df)


    combined = combine_ml_and_msb(ob_df, ob_df['ml_pred'].fillna(1).astype(int).tolist())


    return combined
    

In [132]:
print(build_hybrid_signals(back_df, model, scaler).tail())

                          open       high        low      close     volume  \
timestamp                                                                    
2025-10-10 08:00:00  120932.05  121188.00  120916.44  121159.38  160.19676   
2025-10-10 08:15:00  121159.38  121316.03  121159.37  121279.89   64.03158   
2025-10-10 08:30:00  121279.89  121344.90  121218.66  121278.68   84.83918   
2025-10-10 08:45:00  121278.68  121660.43  121278.68  121631.54  124.04548   
2025-10-10 09:00:00  121631.54  121660.43  121507.68  121554.48   73.16042   

                           RSI          EMA12          EMA26       MACD  \
timestamp                                                                 
2025-10-10 08:00:00  44.956263  121269.572078  121339.851839 -70.279760   
2025-10-10 08:15:00  48.368319  121271.159451  121335.410221 -64.250770   
2025-10-10 08:30:00  48.335920  121272.316458  121331.207982 -58.891524   
2025-10-10 08:45:00  57.315161  121327.581619  121353.454799 -25.873180   
202